# 🚀 Super Hybrid "Agentic" RAG (Upgraded)

Phiên bản này đã được nâng cấp dựa trên các tiêu chuẩn **Production** thực tế, khắc phục các điểm yếu của bản demo trước:

### 🌟 Các Nâng Cấp Đáng Giá (Production-Grade):
1.  **True Agentic Layer (Query Rewriting):** Bổ sung bước "suy luận" chỉnh sửa câu hỏi của user trước khi tìm kiếm, giúp search chính xác hơn với các câu hỏi tối nghĩa.
2.  **SOTA Reranker (Multilingual):** Chuyển sang dùng **`BAAI/bge-reranker-v2-m3`**, model rerank tốt nhất hiện nay cho đa ngôn ngữ (bao gồm tiếng Việt), thay vì model tiếng Anh cũ.
3.  **Robust Fusion (RRF):** Sử dụng thuật toán **Reciprocal Rank Fusion** để gộp kết quả từ Vector và Keyword. Không còn phụ thuộc vào việc "cân đo đong đếm" trọng số alpha (0.5 hay 0.7) nữa.
4.  **Scalar Quantization (Super Fast):** Áp dụng kỹ thuật nén vector (INT8) và Rescore giống `rag_bq` để tăng tốc độ tìm kiếm vectors lên gấp nhiều lần.

---

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install langchain langchain-community langchain-openai qdrant-client rank_bm25 sentence-transformers openai tiktoken

In [1]:
import os
import getpass
from typing import List, Dict, Any, Optional, Tuple

# Setting up OpenAI API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

print("✅ API Key configured!")

Enter your OpenAI API Key:  ········


✅ API Key configured!


## 1. Data Ingestion & Processing
Load và chunking dữ liệu `faq_vneid.txt`.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1. Load Data
file_path = "faq_vneid.txt"
loader = TextLoader(file_path, encoding="utf-8")
raw_documents = loader.load()

# 2. Chunking Strategy
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)

documents = text_splitter.split_documents(raw_documents)
print(f"✅ Loaded chunks: {len(documents)}")

/home/quocanh/.venvs/jupyter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Loaded chunks: 207


## 2. Agentic Layer: Query Rewriting
Trước khi search, Agent sẽ "suy nghĩ" để viết lại câu hỏi sao cho tối ưu nhất cho search engine.

In [3]:
from openai import OpenAI

client = OpenAI()

def rewrite_query(user_query: str) -> str:
    """
    Sử dụng LLM để viết lại câu hỏi của user thành một query search chuẩn.
    Loại bỏ các từ thừa, tập trung vào keyword và ngữ cảnh.
    """
    system_prompt = """
    Bạn là một chuyên gia tối ưu hóa tìm kiếm (SEO) cho hệ thống RAG về VNeID.
    Nhiệm vụ của bạn là viết lại câu hỏi của người dùng thành một câu truy vấn tìm kiếm ngắn gọn, súc tích và chứa nhiều từ khóa quan trọng.
    Giữ nguyên ý nghĩa, chỉ thay đổi cấu trúc để dễ tìm kiếm hơn.
    KHÔNG TRẢ LỜI CÂU HỎI. CHỈ VIẾT LẠI QUERY.
    Ví dụ:
    - User: "cái app này đăng nhập không được, nó báo lỗi tùm lum tà la"
    - Rewrite: "lỗi không đăng nhập được ứng dụng VNeID"
    """
    
    response = client.chat.completions.create(
        model="gpt-4.1-mini", # Hoặc model nhỏ hơn như gpt-3.5-turbo để tiết kiệm/nhanh
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ],
        temperature=0.1
    )
    
    rewritten = response.choices[0].message.content.strip()
    print(f"🔄 Transformed: '{user_query}' -> '{rewritten}'")
    return rewritten

## 3. Core Engine: Hybrid Search (RRF) + SOTA Reranker + Quantization
Nâng cấp: 
- **Reranker**: `BAAI/bge-reranker-v2-m3` (Hỗ trợ tiếng Việt cực tốt).
- **Fusion**: RRF (Reciprocal Rank Fusion) thay vì Weighted Sum.
- **Quantization**: Cấu hình Scalar Quantization (INT8) cho Collection để giảm bộ nhớ và tăng tốc độ search.

In [4]:
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient, models
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
import numpy as np

class SuperRAG_v2:
    def __init__(self, 
                 embedding_model_name="text-embedding-3-large",
                 rerank_model_name="BAAI/bge-reranker-v2-m3", 
                 collection_name="super_rag_v2_quantized"):
        
        # 1. Initialize Embeddings
        self.embeddings = OpenAIEmbeddings(model=embedding_model_name)
        
        # 2. Vector DB (Qdrant In-Memory)
        self.client = QdrantClient(":memory:")
        self.collection_name = collection_name
        self.vector_size = 3072 if "large" in embedding_model_name else 1536
        
        # Check và tạo collection với Quantization Config
        if not self.client.collection_exists(self.collection_name):
             self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=models.VectorParams(size=self.vector_size, distance=models.Distance.COSINE),
                # UPGRADE: Thêm Scalar Quantization để tối ưu tốc độ search (như rag_bq)
                quantization_config=models.ScalarQuantization(
                    scalar=models.ScalarQuantizationConfig(
                        type=models.ScalarType.INT8,
                        quantile=0.99,
                        always_ram=True
                    )
                )
            )
        
        # 3. Reranker Upgrade (COMMENTED OUT FOR SPEED)
        # print(f"⏳ Loading SOTA Reranker: {rerank_model_name}...")
        # self.reranker = CrossEncoder(rerank_model_name)
        # print("✅ Reranker Loaded!")
        self.reranker = None 
        
        self.bm25 = None
        self.documents = []

    def index_data(self, docs: List[Document]):
        print("⏳ Starting Indexing...")
        self.documents = docs
        texts = [doc.page_content for doc in docs]
        
        # A. Sparse Indexing (BM25)
        tokenized_corpus = [text.lower().split() for text in texts]
        self.bm25 = BM25Okapi(tokenized_corpus)
        print("✅ BM25 Sparse Index Created")

        # B. Dense Indexing (Qdrant)
        batch_size = 50
        for i in range(0, len(docs), batch_size):
            batch_docs = docs[i : i + batch_size]
            batch_texts = [d.page_content for d in batch_docs]
            vectors = self.embeddings.embed_documents(batch_texts)
            
            points = [
                models.PointStruct(
                    id=i + idx,
                    vector=vector,
                    payload={"page_content": text, **d.metadata}
                )
                for idx, (text, vector, d) in enumerate(zip(batch_texts, vectors, batch_docs))
            ]
            self.client.upsert(collection_name=self.collection_name, points=points)
            
        print(f"✅ Vector Index Created ({len(docs)} documents)")

    def hybrid_search_rrf(self, query: str, top_k: int = 5, k_rrf: int = 60) -> List[Dict]:
        # 1. Sparse Search (BM25)
        tokenized_query = query.lower().split()
        bm25_indices = np.argsort(self.bm25.get_scores(tokenized_query))[::-1][:50]
        
        # 2. Dense Search (Vector)
        query_vector = self.embeddings.embed_query(query)
        
        # FIX: Dùng query_points và thêm cấu hình SearchParams (Quantization) để search siêu tốc
        search_result = self.client.query_points(
            collection_name=self.collection_name,
            query=query_vector,
            limit=50,
            # UPGRADE: Search với Quantization Search Params (như rag_bq)
            search_params=models.SearchParams(
                quantization=models.QuantizationSearchParams(
                    ignore=False,
                    rescore=True,
                    oversampling=2.0,
                )
            )
        )
        vector_results = search_result.points 
        
        # 3. RRF Fusion
        doc_scores = {}
        for rank, res in enumerate(vector_results):
            idx = res.id
            if idx not in doc_scores:
                doc_scores[idx] = 0.0
            doc_scores[idx] += 1 / (k_rrf + rank + 1)
            
        for rank, idx in enumerate(bm25_indices):
            if idx not in doc_scores:
                doc_scores[idx] = 0.0
            doc_scores[idx] += 1 / (k_rrf + rank + 1)
            
        sorted_doc_ids = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        
        candidates = []
        for idx, score in sorted_doc_ids[:20]: 
            candidates.append({
                "doc": self.documents[idx],
                "rrf_score": score,
                "rerank_score": score # Tạm gán rerank_score bằng rrf_score vì bỏ qua bước rerank
            })
            
        return candidates

    def search(self, query: str, top_k: int = 5) -> List[Dict]:
        candidates = self.hybrid_search_rrf(query)
        if not candidates:
            return []
            
        # COMMENTED OUT RERANKING FOR SPEED
        if self.reranker:
            doc_texts = [c["doc"].page_content for c in candidates]
            pairs = [[query, doc_text] for doc_text in doc_texts]
            
            rerank_scores = self.reranker.predict(pairs)
            
            for i, candidate in enumerate(candidates):
                candidate["rerank_score"] = float(rerank_scores[i])
                
            candidates.sort(key=lambda x: x["rerank_score"], reverse=True)
        
        return candidates[:top_k]

In [5]:
# Khởi tạo hệ thống mới
rag_system = SuperRAG_v2()
rag_system.index_data(documents)

⏳ Starting Indexing...
✅ BM25 Sparse Index Created
✅ Vector Index Created (207 documents)


## 4. Test & Demo (End-to-End Flow)
Chạy thử với quy trình hoàn chỉnh.

In [6]:
import time

def advanced_rag_chat(user_query: str):
    start_time = time.time()
    print(f"\n💬 User: {user_query}")
    
    # 1. Rewriting (Có thể tắt nếu muốn test thuần tốc độ search)
    search_query = rewrite_query(user_query)
    
    # 2. Retrieval & Rerank
    results = rag_system.search(search_query, top_k=3)
    
    end_search_time = time.time()
    print(f"⏱️ Search Time: {end_search_time - start_time:.4f}s")

    print("\n🔍 Top Contexts found:")
    for i, res in enumerate(results):
        print(f"   [{i+1}] Score: {res['rerank_score']:.2f} | ...{res['doc'].page_content[0:100].replace('\n', ' ')}...")
    
    # 3. Generation
    context_str = "\n\n---\n\n".join([c["doc"].page_content for c in results])
    system_prompt = """
    Bạn là trợ lý hỗ trợ VNeID chuyên nghiệp. 
    Hãy trả lời dựa trên context được cung cấp.
    Nếu không có thông tin, hãy thành thật nói không biết.
    """
    
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Question: {user_query}\n\nContext:\n{context_str}"}
        ],
        temperature=0.3
        # streaming=True
    )
    
    print(f"\n🤖 Bot: {response.choices[0].message.content}\n" + "="*40)

# --- TEST CASES ---
test_cases = [
    "tại sao đăng nhập trên thiết bị mới nó bắt nhập mã xác nhận lằng nhằng vậy", # Câu hỏi conversational, cần rewrite
    "làm mất cái mật khẩu của app rồi giờ sao?", 
    "cách đăng ký tài khoản vneid mức 2"
]

for q in test_cases:
    advanced_rag_chat(q)


💬 User: tại sao đăng nhập trên thiết bị mới nó bắt nhập mã xác nhận lằng nhằng vậy
🔄 Transformed: 'tại sao đăng nhập trên thiết bị mới nó bắt nhập mã xác nhận lằng nhằng vậy' -> 'đăng nhập thiết bị mới VNeID yêu cầu nhập mã xác nhận phức tạp'
⏱️ Search Time: 3.4885s

🔍 Top Contexts found:
   [1] Score: 0.03 | ...Sau khi xác nhận thành công, thiết bị cũ sẽ không còn truy cập được vào tài khoản VNeID nữa ạ. 3. Tô...
   [2] Score: 0.03 | ...Sau đó nhập OTP này vào thiết bị B để hoàn tất đăng nhập.  Hoặc anh/chị có thể chọn “Bỏ qua” để đóng...
   [3] Score: 0.03 | ...19. Tại sao hệ thống báo thẻ CCCD đã hết hạn khi đăng nhập? Dạ, đó là khi ngày hết hạn thẻ CCCD của ...

🤖 Bot: Khi đăng nhập trên thiết bị mới, hệ thống yêu cầu nhập mã xác nhận (OTP) nhằm đảm bảo an toàn cho tài khoản của anh/chị. Đây là quy định bảo mật để xác thực rằng chính chủ mới được phép truy cập tài khoản trên thiết bị chưa từng đăng nhập trước đó. Mã OTP sẽ được gửi về số điện thoại đã đăng ký hoặc hiển thị trong ứ